# Tarea 1

José Manuel Jerez Díaz
José Miguel Morales Jure

Implementar la clase `OisCashflow`.

## Configuración Inicial

In [1]:
from finrisk import QC_Financial_3 as Qcf
from dataclasses import dataclass
from enum import Enum
import pandas as pd
from datetime import date
from scipy.optimize import root_scalar

# Modificado por AD
import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

from functools import partial

In [2]:
class BusCal(Enum):
    NY = 1
    SCL = 2

In [3]:
def get_cal(code: BusCal) -> Qcf.BusinessCalendar:
    
    if code == BusCal.NY:
        cal = Qcf.BusinessCalendar(Qcf.QCDate(1, 1, 2020), 20)
        for agno in range(2020, 2071):
            f = Qcf.QCDate(12, 10, agno)
            if f.week_day() == Qcf.WeekDay.SAT:
                cal.add_holiday(Qcf.QCDate(14, 10, agno))
            elif f.week_day() == Qcf.WeekDay.SUN:
                cal.add_holiday(Qcf.QCDate(13, 10, agno))
            elif f.week_day() == Qcf.WeekDay.MON:
                cal.add_holiday(Qcf.QCDate(12, 10, agno))
            elif f.week_day() == Qcf.WeekDay.TUE:
                cal.add_holiday(Qcf.QCDate(11, 10, agno))
            elif f.week_day() == Qcf.WeekDay.WED:
                cal.add_holiday(Qcf.QCDate(10, 10, agno))
            elif f.week_day() == Qcf.WeekDay.THU:
                cal.add_holiday(Qcf.QCDate(9, 10, agno))
            else:
                cal.add_holiday(Qcf.QCDate(8, 10, agno))
        cal.add_holiday(Qcf.QCDate(15, 2, 2021))
        
    return cal

In [4]:
print(BusCal(2))

BusCal.SCL


## `Qcf.time_series`

In [5]:
datos = pd.read_excel("../data/SOFR-10012019-10292020.xls", sheet_name="nice format")
datos=pd.DataFrame(datos)
datos["RATE\n(PERCENT)"]=datos["RATE\n(PERCENT)"]/100
print(datos)

           DATE  BENCHMARK NAME   RATE\n(PERCENT)
0    2020-10-28             SOFR           0.0008
1    2020-10-27             SOFR           0.0009
2    2020-10-26             SOFR           0.0009
3    2020-10-23             SOFR           0.0008
4    2020-10-22             SOFR           0.0007
..          ...              ...              ...
265  2019-10-07             SOFR           0.0183
266  2019-10-04             SOFR           0.0182
267  2019-10-03             SOFR           0.0184
268  2019-10-02             SOFR           0.0185
269  2019-10-01             SOFR           0.0188

[270 rows x 3 columns]


## `Data`

In [6]:
Fechas=[]
for i in range(len(datos["DATE"])):
    año=datos["DATE"][i][0:4]
    año=int(año)
    mes=datos["DATE"][i][5:7]
    mes=int(mes)
    dia=datos["DATE"][i][8:10]
    dia=int(dia)
    fecha= Qcf.QCDate(dia, mes, año)
    Fechas.append(fecha)
datos["Fechas"]=(Fechas)
datos=datos.sort_values(by="DATE",ascending=True)
#print(data["fechas"][1].day_diff(data["fechas"][2]))
datos

,DATE,BENCHMARK NAME,RATE\n(PERCENT),Fechas
269,2019-10-01,SOFR,0.0188,1-10-2019
268,2019-10-02,SOFR,0.0185,2-10-2019
267,2019-10-03,SOFR,0.0184,3-10-2019
266,2019-10-04,SOFR,0.0182,4-10-2019
265,2019-10-07,SOFR,0.0183,7-10-2019
...,...,...,...,...
4,2020-10-22,SOFR,0.0007,22-10-2020
3,2020-10-23,SOFR,0.0008,23-10-2020
2,2020-10-26,SOFR,0.0009,26-10-2020
1,2020-10-27,SOFR,0.0009,27-10-2020


La clase `OisCashflow` es una [`dataclass`](https://realpython.com/python-data-classes/). La tarea consiste en:

- Implementar los 3 métodos de la clase que están con `pass`.
- Implementar la función `present_value`.
- Implementar la función `set_expected_rate`.

In [7]:
datos.rename(columns={"RATE\n(PERCENT)":"fixings"},inplace=True)
datos
fixings=datos['fixings']
fixings
start_date=datos['DATE']
start_date


269    2019-10-01
268    2019-10-02
267    2019-10-03
266    2019-10-04
265    2019-10-07
          ...    
4      2020-10-22
3      2020-10-23
2      2020-10-26
1      2020-10-27
0      2020-10-28
Name: DATE, Length: 270, dtype: object

In [8]:
#Función notebook 2 
def get_wf(start_date: date, fecha_final: date, valor_tasa: float) -> float:
            
    qc_tasa = Qcf.QCInterestRate(valor_tasa, Qcf.QCAct360(), Qcf.QCLinearWf())
    
    return qc_tasa.wf(start_date, fecha_final)

PRUEBA FUNCIONES

Se prueban las funciones por separado para comprobar que funcionan:

In [9]:
def get_accrued_rate(start_date, fixings) -> float:
                
        wf=[]
        final_data=start_date[0]
        for i in range (0,len(start_date)-1):
            
            wf.append(get_wf(start_date[269-i],start_date[268-i],fixings[i]))
            
        wf.append(get_wf(start_date[1],start_date[0],fixings[0]))
        df=pd.DataFrame()
        df["df"]=(wf)
        P=df["df"].product()
        #print(P)
        dias = start_date[269].day_diff(start_date[0])
# Tasa equivalente
# prod = 1 + tasa_eq * dias / 360
        tasa_eq = (P-1) * 360.0 / dias
        return tasa_eq
get_accrued_rate(datos["Fechas"], datos["fixings"])


0.00696242230123633

In [10]:
  def get_accrued_interest(start_date, fixings) -> float:
        
        nocional=1000000
        tasa_eq=get_accrued_rate(start_date, fixings) 
        dias = start_date[269].day_diff(start_date[0])
        Interes=(nocional * tasa_eq * dias) / 360
        return Interes
 
 
get_accrued_interest(datos["Fechas"], datos["fixings"])

7600.644345516327

In [11]:
  def amount( start_date,fixings) -> float:
        """
        Calcula el flujo total al vencimiento (amortización más intereses devengados hasta self.end_date).
        Los valores de las  tasas overnight deben estar almacenados en la variable `fixings`.
        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
    
        
        nocional=1000000
        Interes=get_accrued_interest(start_date,fixings)
        dias = start_date[269].day_diff(start_date[0])
        Flujo_total=nocional+ Interes
        
        return Flujo_total
c=amount(datos["Fechas"],datos["fixings"])
print(c)

1007600.6443455163


In [12]:
@dataclass # syntactic sugar
class OisCashflow:
    start_date: Qcf.QCDate
    end_date: Qcf.QCDate
    settlement_date: Qcf.QCDate
    notional: float
    currency: Qcf.QCCurrency
    amortization: float
    amort_is_cashflow: bool
    interest_rate: Qcf.QCInterestRate
    on_index: Qcf.InterestRateIndex
    spread: float
    gearing: float
        
    
        
    def get_accrued_rate(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
        
        """
        Calcula la tasa equivalente desde `self. start_date` a `accrual_date`. La tasa equivalente
        se calcula como:
        
        (P - 1) * 360 / (accrual_date - self.start_date)
        
        donde P es el producto de los factores de capitalización de todas las tasas overnight
        entre `start_date` y `accrual_date`. Los valores de esas tasas deben estar almacenados
        en la variable `fixings`.
        tasa_lin_act360 = Qcf.QCInterestRate(.1, act360, lin_wf)
        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
                        
        """
        # Factor de capitalización
        wf=[]
        self.final_data=start_date[0]
        
        for i in range (0,len(self.final_data)-1):
            
            wf.append(get_wf(self.final_data[269-i],self.final_data[268-i],fixings[i]))
            
        wf.append(get_wf(self.final_data[1],self.final_data[0],fixings[0]))
        dataf=pd.DataFrame()
        dataf["df"]=(wf)
        # Calculo del producto de Factor de capitalización
        P=dataf["df"].product()
        dias = self.final_data[269].day_diff(self.final_data[0])
# Tasa equivalente
# prod = 1 + tasa_eq * dias / 360
        tasa_eq = (P-1) * 360.0 / dias
        #return tasa_eq,P
        return tasa_eq
       
     
    
    def get_accrued_interest(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
        """
        Calcula el interés (plata) devengado desde `self.start_date` a `accrual_date` utilizando la tasa equivalente
        que se calcula con el método anterior.
        
        Los valores de las  tasas overnight deben estar almacenados en la variable `fixings`.
        
        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
        #Nocional va en class
        #nocional=1000000
        tasa_eq=get_accrued_rate(start_date, fixings) 
        
        dias = start_date[269].day_diff(start_date[0])
        
        Interes=(nocional * tasa_eq * dias) / 360
        return Interes,tasa_eq
 
    pass
    
    def amount(self, fixings: Qcf.time_series) -> float:
        """
        Calcula el flujo total al vencimiento (amortización más intereses devengados hasta self.end_date).
        
        Los valores de las  tasas overnight deben estar almacenados en la variable `fixings`.
        
        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
         #Nocional lo entrega el class
        #nocional=1000000
        Interes=get_accrued_interest(start_date,fixings)
        dias = start_date[269].day_diff(start_date[0])
        Flujo_total=nocional+ Interes
        
        return Flujo_total
    
    pass
    


Construcción de un objeto `Qcf.InterestRateIndex`.

In [13]:
codigo = 'SOFR'
tasa_on = Qcf.QCInterestRate(.0, Qcf.QCAct360(), Qcf.QCLinearWf())
fixing_lag = Qcf.Tenor('0d')
tenor = Qcf.Tenor('1d')
fixing_calendar = get_cal(BusCal.NY)     
settlement_calendar = fixing_calendar  
sofr = Qcf.InterestRateIndex(
    codigo,
    tasa_on,
    fixing_lag,
    tenor,
    fixing_calendar,
    settlement_calendar,
    Qcf.QCUSD()
)

Construcción de una instancia de `OisCashflow`.

In [14]:
ois = OisCashflow(
    Qcf.QCDate(26, 12, 2019),
    Qcf.QCDate(26, 1, 2020),
    Qcf.QCDate(26, 1, 2020),
    10000000,
    Qcf.QCUSD(),
    1000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1
)

In [15]:
ois.get_accrued_rate(ois.end_date, fixings)

IndexError: string index out of range

In [16]:
print(ois.start_date)

26-12-2019


In [17]:
print(ois.notional)

10000000


## Funciones

In [18]:
datos3 = pd.read_excel("../data/20201012_built_sofr_zero.xlsx")
datos3


,plazo,tasa,df
0,1,0.000811,0.999998
1,7,0.000841,0.999984
2,14,0.000780,0.999970
3,21,0.000774,0.999955
4,33,0.000781,0.999929
5,61,0.000781,0.999870
6,92,0.000811,0.999796
7,125,0.000781,0.999733
8,152,0.000760,0.999683
9,182,0.000750,0.999626


In [19]:
fixings=datos3["tasa"]
fixings
frmt = {
    'tasa': '{:.6%}',
    'df': '{:.6%}',
    'valor_tasa': '{:.4%}',
    'spread': '{:.4%}',
    'nominal': '{:,.2f}',
    'interes': '{:,.2f}',
    'amortizacion': '{:,.2f}',
    'flujo': '{:,.2f}',
}

In [20]:
def op(rp,notional,start_date,tenor,fixed_rate_value,spread,gearing):
    op = get_ois_sofr(
    rp=Qcf.RecPay.RECEIVE,
    notional=10000000,
    start_date=Qcf.QCDate(31, 10, 2020),
    tenor=Qcf.Tenor('1Y'),
    fixed_rate_value=.01,
    spread=0.0,
    gearing=1.0)
    return op

In [21]:
def set_expected_rate(
        val_date: Qcf.QCDate,
        ois: OisCashflow ,
        zcc: Qcf.ZeroCouponCurve) -> None:
    """
    Esta función opera de la misma forma que la análoga función de `QC_Financial_3`.
    Ver por ejemplo los casos cuando usamos el objeto Qcf.ForwardRates() en
    el notebook 9.
    """
    zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), datos3)       
    get_ois_sofr = partial(aux.get_ois_using_template, aux.type_ois_template, aux.TypeOis.SOFR)
     
    operacion=op(Qcf.RecPay.RECEIVE,10000000,Qcf.QCDate(31, 10, 2020),Qcf.Tenor('1Y'),0.01,0.0,1.0)
    forward = Qcf.ForwardRates()
    
    for t in fixings:
        Resultados = forward.set_rates_icp_clp_leg(val_date, t, operacion[1], zcc)
        return aux.show_leg(operacion[1], 'IcpClpCashflow', '').style.format(frmt)
    
    
    # Para un OIS calculando en t0 un cashflow entre (t0 <) t1 < t2
    # ICP(t0)*wf(to, t2) = ICP(t2), ICP(t0)*w(t0, t1)=ICP(t1) -> ICP(t2)/ICP(t1)=ICP(t1,t2)
    # tasa(t1, t2) = [ICP(t1, t2) - 1]*360 / (t2-t1)
    # El ICP se puede pensar como el account asociado a la tasa interbancaria chilena,
    # que es el índice overnight chileno.
    #
    # Cuando t1 < t0 < t2
    #tener un poquito de ojo ... porque el cashflow ya tiene fixings.


In [22]:
set_expected_rate(Qcf.QCDate(28,10,2020),get_ois_sofr,zcc)

NameError: name 'get_ois_sofr' is not defined

Prueba Funciones:

In [23]:
zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), datos3)
print(zcc)
plazo = 365
print(f"Tasa esperada a {plazo} días es igual a {zcc.get_rate_at(plazo):.4%}")
print(f"Factor de descuento a {plazo} días es igual a {zcc.get_discount_factor_at(plazo):.6%}")

get_ois_sofr = partial(aux.get_ois_using_template, aux.type_ois_template, aux.TypeOis.SOFR)


Tasa esperada a 365 días es igual a 0.0702%
Factor de descuento a 365 días es igual a 99.929812%


In [24]:
operacion=op(Qcf.RecPay.RECEIVE,10000000,Qcf.QCDate(31, 10, 2020),Qcf.Tenor('1Y'),0.01,0.0,1.0)
vp = Qcf.PresentValue()
fecha_val = Qcf.QCDate(28 , 10, 2020)
vp_fija = vp.pv(fecha_val, operacion[0], zcc)
print(f'El valor presente de la pata fija es: USD {vp_fija:,.2f}')

El valor presente de la pata fija es: USD 10,093,727.67


In [25]:
#Valor presente pata flotante (Notebook 9)
fwd = Qcf.ForwardRates()
fwd.set_rates_icp_clp_leg(fecha_val, 1.0, operacion[1], zcc)
vp_flot = vp.pv(fecha_val, operacion[1], zcc)
print(f'El valor presente de la pata flotante es: USD {vp_flot:,.2f}')
print(f'Por lo tanto, el valor total de la operación es:\nValor total: USD {vp_fija + vp_flot:,.2f}')

El valor presente de la pata flotante es: USD -9,999,932.56
Por lo tanto, el valor total de la operación es:
Valor total: USD 93,795.12


In [26]:
def present_value(val_date: Qcf.QCDate,
                  ois_cashflow: OisCashflow,
                  zcc: Qcf.ZeroCouponCurve) -> float:
    
        
    zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), datos3)
    print(zcc)
    plazo = 365
    Tasa_esperada=zcc.get_rate_at(plazo)
    factor_descuento=zcc.get_discount_factor_at(plazo)

    get_ois_sofr = partial(aux.get_ois_using_template, aux.type_ois_template, aux.TypeOis.SOFR)
    
    operacion=op(Qcf.RecPay.RECEIVE,10000000,Qcf.QCDate(31, 10, 2020),Qcf.Tenor('1Y'),0.01,0.0,1.0)
    
    vp = Qcf.PresentValue()
    fecha_val = Qcf.QCDate(28, 10, 2020)
    vp_fija = vp.pv(fecha_val, operacion[0], zcc)
    
    fwd = Qcf.ForwardRates()
    fwd.set_rates_icp_clp_leg(fecha_val, 1.0, operacion[1], zcc)
    vp_flot = vp.pv(fecha_val, operacion[1], zcc)
     
    return vp_fija+vp_flot
    
    
    """
    Esta función opera de la misma forma que la análoga función de `QC_Financial_3`.
    Para probar esta función utilizar la data de que está en data/20201012_built_sofr_zero.xlsx.
    Para traer a valor presente debes respetar la convención de las tasas de la curva.
    
    NOTA: la variable ois_cashflow debe haber 'pasado' por la función set_expected_rate para que
    tenga los intereses de los fixings futuros.
    """
   

In [27]:
present_value(Qcf.QCDate(28,10,2020), get_ois_sofr,zcc)


93795.11645178124

## Tests

In [28]:
import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

In [29]:
amount_tol = 10000
rate_tol = .001

In [30]:
exitos = 0

In [31]:
def suma_exito(resultado, check, tipo):
    global exitos
    if resultado is None:
        return
    else:
        print('Suma medio punto por obtener resultado.')
        exitos += .5
        if tipo == 'monto':
            if abs(resultado - check) < amount_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1
        else:
            if abs(resultado - check) < rate_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1

Construcción de una instancia de `OisCashflow`.

In [32]:
ois = OisCashflow(
    Qcf.QCDate(1, 10, 2019),
    Qcf.QCDate(1, 10, 2020),
    Qcf.QCDate(1, 10, 2020),
    10000000,
    Qcf.QCUSD(),
    1000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1
)

En la siguiente variable `exitos` se registra cuantos tests se superan con éxito.

### Objeto `fixings`

In [33]:
df_fixings = pd.read_excel('../data/SOFR-10012019-10292020.xls', sheet_name='nice format')
df_fixings.columns = ['fecha', 'nombre', 'valor']
df_fixings['valor'] /= 100

In [34]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


In [35]:
fixings = Qcf.time_series()
for row in df_fixings.itertuples():
    fixings[Qcf.build_qcdate_from_string(row.fecha)] = row.valor

In [36]:
fixings[Qcf.QCDate(13, 1, 2020)]

0.0154

### `OisCashflow.accrued_rate`

In [37]:
accrued_rate = ois.get_accrued_rate(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued rate: {accrued_rate:.8%}')

IndexError: string index out of range

AD: test entregado por el grupo

In [38]:
# Ejecutar accrued_rate no como Clase, sino como una función a parte, además en Fechas vienen como QCDate
accrued_rate = get_accrued_rate(datos["Fechas"], datos["fixings"])
print(f'accrued rate: {accrued_rate:.8%}')
#Resultado tarea:0.69624223%

accrued rate: 0.69624223%


Se verificará el cálculo usando `df_fixings`.

In [39]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


Se agrega la columna `next_date`, servirá para calcular el factor de capitalización de cada tasa.

In [40]:
def next_date(fecha:str, calendario: Qcf.BusinessCalendar) -> str:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    next_fecha = calendario.shift(qfecha, 1)
    return next_fecha.description(False)

In [41]:
df_fixings['next_fecha'] = df_fixings.apply(
    lambda row: next_date(row['fecha'], settlement_calendar),
    axis=1
)

In [42]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha
0,2020-10-28,SOFR,0.0800%,2020-10-29
1,2020-10-27,SOFR,0.0900%,2020-10-28
2,2020-10-26,SOFR,0.0900%,2020-10-27
3,2020-10-23,SOFR,0.0800%,2020-10-26
4,2020-10-22,SOFR,0.0700%,2020-10-23


Se calcula ahora el factor de capitalización.

In [43]:
def factor_cap(fecha: str, next_fecha: str, valor: float) -> float:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    qnext_fecha = Qcf.build_qcdate_from_string(next_fecha)
    int_rate = Qcf.QCInterestRate(valor, Qcf.QCAct360(), Qcf.QCLinearWf())
    return int_rate.wf(qfecha, qnext_fecha)

In [44]:
df_fixings['factor_capitalizacion'] = df_fixings.apply(
    lambda row: factor_cap(row['fecha'], row['next_fecha'], row['valor']),
    axis=1
)

In [45]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha,factor_capitalizacion
0,2020-10-28,SOFR,0.0800%,2020-10-29,1.000002
1,2020-10-27,SOFR,0.0900%,2020-10-28,1.000002
2,2020-10-26,SOFR,0.0900%,2020-10-27,1.000002
3,2020-10-23,SOFR,0.0800%,2020-10-26,1.000007
4,2020-10-22,SOFR,0.0700%,2020-10-23,1.000002


Veamos donde están en `df_fixings` los valores para `start_date` de `ois` y la fecha 15-6-2020 y calculamos el producto de los factores de capitalización desde esa fecha hasta la última fecha de `df_flujos` (que es el primer registro). Luego, el cociente de los dos factores nos dará el factor entre ambas fechas. Finalmente, con ese factor, se calculará `accrued_rate`.

In [46]:
start_date = ois.start_date.description(False)
df_fixings[df_fixings.fecha == start_date]

,fecha,nombre,valor,next_fecha,factor_capitalizacion
269,2019-10-01,SOFR,0.0188,2019-10-02,1.000052


In [47]:
factor_largo = df_fixings.iloc[:270]['factor_capitalizacion'].prod()
print(factor_largo)
qstart_date = Qcf.build_qcdate_from_string('2019-10-01')

1.0075715527341267


In [48]:
df_fixings[df_fixings.fecha == '2020-06-15']

,fecha,nombre,valor,next_fecha,factor_capitalizacion
94,2020-06-15,SOFR,0.0009,2020-06-16,1.000002


In [49]:
factor_corto = df_fixings.iloc[:95]['factor_capitalizacion'].prod()
qaccrued_date = Qcf.build_qcdate_from_string('2020-06-15')

In [50]:
dias = qstart_date.day_diff(qaccrued_date)
check_accrued_rate = (factor_largo / factor_corto - 1.0) * 360.0 / dias
print(f'La diferencia es: {accrued_rate - check_accrued_rate:.8%}')

La diferencia es: -0.31367091%


In [51]:
suma_exito(accrued_rate, check_accrued_rate, 'tasa')

Suma medio punto por obtener resultado.


### `OisCashflow.accrued_interest`

In [52]:
accrued_interest = ois.get_accrued_interest(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued interest: {accrued_interest:,.2f}')

IndexError: string index out of range

AD: test entregado por el grupo.

In [53]:
#Ejecutar accrued_interest de la misma forma que función anterior
accrued_interest = get_accrued_interest(datos["Fechas"], datos["fixings"])
print(f'accrued interest: {accrued_interest:,.2f}')
#Resultado tarea: 7,600.64

accrued interest: 7,600.64


In [54]:
ois.notional

10000000

In [55]:
check_accrued_interest = ois.notional * accrued_rate * dias / 360.0
print(f'La diferencia es: {accrued_interest - check_accrued_interest:,.6f}')

La diferencia es: -42,296.715480


In [56]:
suma_exito(accrued_interest, check_accrued_interest, 'monto')

Suma medio punto por obtener resultado.


### `OisCashflow.amount`

In [57]:
amount = ois.amount(fixings)
print(f'amount: {amount:,.2f}')

IndexError: string index out of range

AD: aplica test entregado por el grupo.

In [58]:
result_amount = amount(datos["Fechas"],datos["fixings"])
#Resultado tarea:1007600.6443455163

In [59]:
end_date = ois.end_date.description(False)
print(f'end date: {end_date}')
df_fixings[df_fixings.fecha == end_date]

end date: 2020-10-01


,fecha,nombre,valor,next_fecha,factor_capitalizacion
18,2020-10-01,SOFR,0.0008,2020-10-02,1.000002


In [60]:
factor_corto_2 = df_fixings.iloc[:19]['factor_capitalizacion'].prod()
dias_2 = qstart_date.day_diff(ois.end_date)
accrued_rate_amount = (factor_largo / factor_corto_2- 1.0) * 360.0 / dias_2
check_amount = ois.notional * accrued_rate_amount * dias_2 / 360.0 + ois.amortization
print(f'La diferencia es: {result_amount - check_amount:.6f}')

La diferencia es: -67426.401088


In [61]:
suma_exito(result_amount, check_amount, 'monto')

Suma medio punto por obtener resultado.


### `set_expected_rate`

Comenzamos cargando los valores de la curva cero cupón que se utilizará para hacer pruebas de `set_expected_rate` y `present_value`.

In [62]:
df_curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')

In [63]:
df_curva.head().style.format({'tasa': '{:.4%}', 'df': '{:.6%}'})

,plazo,tasa,df
0,1,0.0811%,99.999778%
1,7,0.0841%,99.998388%
2,14,0.0780%,99.997010%
3,21,0.0774%,99.995549%
4,33,0.0781%,99.992942%


Con la data se construye un objeto de tipo `Qcf.ZeroCouponCurve`.

In [64]:
zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)

Comienzan los tests.

In [65]:
print(f'Fecha inicial flujo: {ois.start_date.description(False)}')
print(f'Fecha final flujo: {ois.end_date.description(False)}')

Fecha inicial flujo: 2019-10-01
Fecha final flujo: 2020-10-01


Se modifican según los tests entregados por el grupo.

In [66]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, get_ois_sofr, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

ArgumentError: Python argument types in
    None.set_rates_icp_clp_leg(ForwardRates, QCDate, map_indexing_suite_time_series_entry, Leg, ZeroCouponCurve)
did not match C++ signature:
    set_rates_icp_clp_leg(QCode::Financial::ForwardRates {lvalue}, QCDate, double, QCode::Financial::Leg {lvalue}, QCode::Financial::ZeroCouponCurve {lvalue})

In [67]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, get_ois_sofr, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

ArgumentError: Python argument types in
    None.set_rates_icp_clp_leg(ForwardRates, QCDate, map_indexing_suite_time_series_entry, Leg, ZeroCouponCurve)
did not match C++ signature:
    set_rates_icp_clp_leg(QCode::Financial::ForwardRates {lvalue}, QCDate, double, QCode::Financial::Leg {lvalue}, QCode::Financial::ZeroCouponCurve {lvalue})

In [68]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, get_ois_sofr, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = ois.start_date.day_diff(val_date)
p2 = val_date.day_diff(ois.end_date)
wf1 = 1 + ois.get_accrued_interest(val_date, fixings) / ois.notional
wf2 = 1 / zcc.get_discount_factor_at(p2)
check = (wf1 * wf2 - 1) * 360 / (p1 + p2)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

ArgumentError: Python argument types in
    None.set_rates_icp_clp_leg(ForwardRates, QCDate, map_indexing_suite_time_series_entry, Leg, ZeroCouponCurve)
did not match C++ signature:
    set_rates_icp_clp_leg(QCode::Financial::ForwardRates {lvalue}, QCDate, double, QCode::Financial::Leg {lvalue}, QCode::Financial::ZeroCouponCurve {lvalue})

In [69]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, get_ois_sofr, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(val_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

ArgumentError: Python argument types in
    None.set_rates_icp_clp_leg(ForwardRates, QCDate, map_indexing_suite_time_series_entry, Leg, ZeroCouponCurve)
did not match C++ signature:
    set_rates_icp_clp_leg(QCode::Financial::ForwardRates {lvalue}, QCDate, double, QCode::Financial::Leg {lvalue}, QCode::Financial::ZeroCouponCurve {lvalue})

In [70]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, get_ois_sofr, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(ois.end_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

ArgumentError: Python argument types in
    None.set_rates_icp_clp_leg(ForwardRates, QCDate, map_indexing_suite_time_series_entry, Leg, ZeroCouponCurve)
did not match C++ signature:
    set_rates_icp_clp_leg(QCode::Financial::ForwardRates {lvalue}, QCDate, double, QCode::Financial::Leg {lvalue}, QCode::Financial::ZeroCouponCurve {lvalue})

### `present_value`

In [71]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, get_ois_sofr, zcc)
pv = present_value(val_date, get_ois_sofr, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

ArgumentError: Python argument types in
    None.set_rates_icp_clp_leg(ForwardRates, QCDate, map_indexing_suite_time_series_entry, Leg, ZeroCouponCurve)
did not match C++ signature:
    set_rates_icp_clp_leg(QCode::Financial::ForwardRates {lvalue}, QCDate, double, QCode::Financial::Leg {lvalue}, QCode::Financial::ZeroCouponCurve {lvalue})

In [72]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, get_ois_sofr, zcc)
pv = present_value(val_date, get_ois_sofr, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

ArgumentError: Python argument types in
    None.set_rates_icp_clp_leg(ForwardRates, QCDate, map_indexing_suite_time_series_entry, Leg, ZeroCouponCurve)
did not match C++ signature:
    set_rates_icp_clp_leg(QCode::Financial::ForwardRates {lvalue}, QCDate, double, QCode::Financial::Leg {lvalue}, QCode::Financial::ZeroCouponCurve {lvalue})

In [73]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, get_ois_sofr, zcc)
pv = present_value(val_date, get_ois_sofr, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

ArgumentError: Python argument types in
    None.set_rates_icp_clp_leg(ForwardRates, QCDate, map_indexing_suite_time_series_entry, Leg, ZeroCouponCurve)
did not match C++ signature:
    set_rates_icp_clp_leg(QCode::Financial::ForwardRates {lvalue}, QCDate, double, QCode::Financial::Leg {lvalue}, QCode::Financial::ZeroCouponCurve {lvalue})

In [74]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, get_ois_sofr, zcc)
pv = present_value(val_date, get_ois_sofr, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

ArgumentError: Python argument types in
    None.set_rates_icp_clp_leg(ForwardRates, QCDate, map_indexing_suite_time_series_entry, Leg, ZeroCouponCurve)
did not match C++ signature:
    set_rates_icp_clp_leg(QCode::Financial::ForwardRates {lvalue}, QCDate, double, QCode::Financial::Leg {lvalue}, QCode::Financial::ZeroCouponCurve {lvalue})

In [75]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, get_ois_sofr, zcc)
pv = present_value(val_date, get_ois_sofr, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

ArgumentError: Python argument types in
    None.set_rates_icp_clp_leg(ForwardRates, QCDate, map_indexing_suite_time_series_entry, Leg, ZeroCouponCurve)
did not match C++ signature:
    set_rates_icp_clp_leg(QCode::Financial::ForwardRates {lvalue}, QCDate, double, QCode::Financial::Leg {lvalue}, QCode::Financial::ZeroCouponCurve {lvalue})

In [76]:
print(f'Éxitos totales: {exitos}')

Éxitos totales: 1.5
